In [ ]:
input_string = """
(<1> [topic] (자율주행 자동차는 언제 출시될 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {44445} <인게임 2번유저> (주제 대화2) 타임스탬프2 | {11115} <인게임 3번유저> (주제 대화3) 타임스탬프3 | {11115} <인게임 3번유저> (주제 대화4) 타임스탬프4 | {12345} <인게임 1번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {44445} <인게임 2번유저> (자유 대화1) 타임스탬프1 | {11115} <인게임 3번유저> (자유 대화2) 타임스탬프2 | [event] {vote} <인게임 1번유저> (타겟1) 타임스탬프1 | ), 
(<2> [topic] (자율주행 자동차는 언제 고장날 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {12345} <인게임 1번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {11115} <인게임 3번유저> (자유 대화1) 타임스탬프2 | [event] {vote} <인게임 4번유저> (타겟2) 타임스탬프1 | ), 
(<3> [topic] (자율주행 자동차는 언제 고장날 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {11115} <인게임 3번유저> (주제 대화2) 타임스탬프2 | {11115} <인게임 3번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {98765} <인게임 6번유저> (자유 대화1) 타임스탬프2 | [event] {vote} <인게임 5번유저> (타겟6) 타임스탬프1 | )
"""

In [72]:
import re
import json


def parse_input_string(input_string):
    round_list = input_string.strip()[1:-1].split("), (")
    round_dict = {}

    for round_str in round_list:
        round_num = round_str[1]
        round_dialogue = round_str.strip()[4:-1]
        round_dict[round_num] = {}

        # [topic] 섹션 파싱
        topic_pattern = r"\[topic\]\s*\((.*?)\)"
        topic_match = re.search(topic_pattern, round_dialogue)
        if topic_match:
            round_dict[round_num]["topic"] = topic_match.group(1)

        # [topic_debate] 섹션 파싱
        topic_debate_pattern = r"\[topic_debate\](.*?)\[free_debate\]"
        topic_debate_match = re.search(topic_debate_pattern, round_dialogue)
        if topic_debate_match:
            debates = topic_debate_match.group(1).strip().split("|")
            parsed_debates = []
            for debate in debates:
                debate = debate.strip()
                if debate:
                    user_pattern = r"\{(\d+)\}\s*<[^>]+>\s*\(([^)]+)\)"
                    user_match = re.search(user_pattern, debate)
                    if user_match:
                        parsed_debates.append(
                            {
                                "user_id": user_match.group(1),
                                "content": user_match.group(2),
                            }
                        )
            round_dict[round_num]["topic_debate"] = parsed_debates

        # [free_debate] 섹션 파싱
        free_debate_pattern = r"\[free_debate\](.*?)\[event\]"
        free_debate_match = re.search(free_debate_pattern, round_dialogue)
        if free_debate_match:
            debates = free_debate_match.group(1).strip().split("|")
            parsed_debates = []
            for debate in debates:
                debate = debate.strip()
                if debate:
                    user_pattern = r"\{(\d+)\}\s*<[^>]+>\s*\(([^)]+)\)"
                    user_match = re.search(user_pattern, debate)
                    if user_match:
                        parsed_debates.append(
                            {
                                "user_id": user_match.group(1),
                                "content": user_match.group(2),
                            }
                        )
            round_dict[round_num]["free_debate"] = parsed_debates

    return json.dumps(round_dict, ensure_ascii=False, indent=2)

In [74]:
result = [parse_input_string(input_string)]
print(result)

['{\n  "1": {\n    "topic": "자율주행 자동차는 언제 출시될 것인가?",\n    "topic_debate": [\n      {\n        "user_id": "12345",\n        "content": "주제 대화1"\n      },\n      {\n        "user_id": "44445",\n        "content": "주제 대화2"\n      },\n      {\n        "user_id": "11115",\n        "content": "주제 대화3"\n      },\n      {\n        "user_id": "11115",\n        "content": "주제 대화4"\n      },\n      {\n        "user_id": "12345",\n        "content": "마지막_대화"\n      }\n    ],\n    "free_debate": [\n      {\n        "user_id": "44445",\n        "content": "자유 대화1"\n      },\n      {\n        "user_id": "11115",\n        "content": "자유 대화2"\n      }\n    ]\n  },\n  "2": {\n    "topic": "자율주행 자동차는 언제 고장날 것인가?",\n    "topic_debate": [\n      {\n        "user_id": "12345",\n        "content": "주제 대화1"\n      },\n      {\n        "user_id": "12345",\n        "content": "마지막_대화"\n      }\n    ],\n    "free_debate": [\n      {\n        "user_id": "11115",\n        "content": "자유 대화1"\n      }\n    ]\n  },\